In [ ]:
import requests
import pandas as pd
import time
import json
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# ---------- TIME ZONE ----------
try:
    from zoneinfo import ZoneInfo
    ET_TZ = ZoneInfo("America/New_York")
    BERLIN_TZ = ZoneInfo("Europe/Berlin")
except Exception:
    ET_TZ = None
    BERLIN_TZ = None

# ---------- HEADERS ----------
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

# ---------- SPORT SLUG ----------
SPORT_SLUG = {
    "Men": "mens-college-basketball",
    "Women": "womens-college-basketball"
}


def parse_time_to_berlin(date_str, time_str):
    """Convert ET to Berlin time. If not a valid time, return original."""
    if ET_TZ is None or BERLIN_TZ is None:
        return time_str

    txt = time_str.lower()
    if any(x in txt for x in ["final", "tbd", "post", "ppd"]):
        return time_str

    try:
        dt_et = datetime.strptime(f"{date_str} {time_str}", "%Y%m%d %I:%M %p")
        dt_et = dt_et.replace(tzinfo=ET_TZ)
        dt_berlin = dt_et.astimezone(BERLIN_TZ)
        return dt_berlin.strftime("%Y-%m-%d %H:%M")
    except Exception:
        return time_str


# ---------- NEW: API VENUE FETCH ----------
def fetch_venue_from_api(game_url):
    """
    Fetch venue details from ESPN official API.
    Example:
    https://site.web.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/summary?event=401688813
    """
    if not game_url:
        return ""

    try:
        # pull the event id (digits in URL)
        import re
        m = re.search(r'gameId/(\d+)', game_url)
        if not m:
            return ""

        event_id = m.group(1)

        api_url = f"https://site.web.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/summary?event={event_id}"
        r = requests.get(api_url, headers=HEADERS, timeout=30)
        data = r.json()

        venue = data.get("gameInfo", {}).get("venue", {})

        name = venue.get("fullName", "")
        addr = venue.get("address", {})

        city = addr.get("city", "")
        state = addr.get("state", "")
        country = addr.get("country", "")

        parts = [p for p in [name, city, state, country] if p]
        return ", ".join(parts)

    except Exception:
        return ""


# ---------- HTML FALLBACK ----------
def fetch_venue_html(game_url):
    """Fallback to HTML scraping if API fails."""
    try:
        r = requests.get(game_url, headers=HEADERS, timeout=30)
        soup = BeautifulSoup(r.text, "html.parser")

        el = soup.find("span", attrs={"data-testid": "game-location"})
        if el:
            return el.get_text(" ", strip=True)

    except Exception:
        pass

    return ""


def fetch_venue(game_url):
    """Safe wrapper combining API + HTML fallback."""
    venue = fetch_venue_from_api(game_url)

    if not venue:
        venue = fetch_venue_html(game_url)

    return venue


def fetch_espn_schedule(date, sport_slug):
    """Fetch schedule for ONE date."""
    url = f"https://www.espn.com/{sport_slug}/schedule/_/date/{date}"
    r = requests.get(url, headers=HEADERS, timeout=30)
    soup = BeautifulSoup(r.text, "html.parser")

    rows = soup.select("table tbody tr")
    if not rows:
        return pd.DataFrame()

    fixtures = []

    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 3:
            continue

        away = cols[0].get_text(" ", strip=True)
        home = cols[1].get_text(" ", strip=True)
        time_status = cols[2].get_text(" ", strip=True)

        if not away or not home:
            continue

        # ---- GAME URL ----
        game_url = ""
        for a in row.find_all("a", href=True):
            if "gameId" in a["href"]:
                href = a["href"]
                if href.startswith("/"):
                    href = "https://www.espn.com" + href
                game_url = href
                break

        # ---- VENUE (API first, HTML fallback) ----
        venue = fetch_venue(game_url)
        time.sleep(0.2)

        fixtures.append({
            "Date (ET)": date,
            "Away Team": away,
            "Home Team": home,
            "Time / Status (ET)": time_status,
            "Time (Berlin)": parse_time_to_berlin(date, time_status),
            "Venue": venue,
            "Game URL": game_url
        })

    return pd.DataFrame(fixtures)


def extract_schedule_to_excel(start_date, end_date, sport, output_file):
    """Extract date range and save to Excel."""
    sport_slug = SPORT_SLUG[sport]

    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    all_data = []
    current = start

    while current <= end:
        date_str = current.strftime("%Y%m%d")
        print(f"\n📅 Fetching {date_str} ({sport}) ...")

        df = fetch_espn_schedule(date_str, sport_slug)

        if df.empty:
            print("⚠ No games found.")
        else:
            print(f"✔ {len(df)} games found.")
            all_data.append(df)

        time.sleep(0.5)
        current += timedelta(days=1)

    if not all_data:
        print("\n❌ No fixtures for the range.")
        return pd.DataFrame()

    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_excel(output_file, index=False)
    print(f"\n🎯 Excel created: {output_file}")

    return final_df


# ---------- RUN ----------
if __name__ == "__main__":
    df = extract_schedule_to_excel(
        start_date="2025-11-03",
        end_date="2026-03-15",
        sport="Men",     # change to "Women" if needed
        output_file="NCAA_Fixtures_With_Venue_1.xlsx"
    )

    print(df.head())
